In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [4]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std*nstd
    lower_band = sma - std*nstd
    return upper_band, lower_band


symbols = ['BTC', 'ETH']


nstd = 2

for symbol in symbols:
    df[f'{symbol}_SMA'] = sma(df[f'{symbol}-USD_Open'], 20)
    df[f'{symbol}_Upper_Band'], df[f'{symbol}_Lower_Band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_SMA'], 20, nstd)


df.dropna(inplace=True)

In [610]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.short_amount = 0
        self.short_price = 0
        self.short_unit = None
        self.band_stop_run = 0.999     # <= 1

        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.symbols = symbols
        
        self.buys = []
        self.sells = []
        
        self.trading_fee_multiplier = 1      # VIP level 0, paying fees with BNB 0.99975
        self.number_of_buys = len(self.buys)
        self.sl = 2

        self.cross_up = {}
        self.cross_down = {}
        self.reset_cross_up([symbols])
        self.reset_cross_down([symbols])

    def buy_long(self, symbol, long_price, time):
        self.balance_amount = (self.balance_amount / long_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, long_price, 'long'])
        self.number_of_buys = len(self.buys)
        print('filled long')

    def buy_short(self, symbol, short_price, time):
        self.short_amount = (self.balance_amount / short_price) * self.trading_fee_multiplier
        self.short_price = short_price
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price, 'short'])
        self.number_of_buys = len(self.buys)
        print('filled short')

    def sell_long(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'
    
    def sell_short(self, sell_price, time):
        self.balance_amount = self.balance_amount + (self.short_amount * (self.short_price - sell_price) * self.trading_fee_multiplier)
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0
    

    def reset_cross_up(self, symbols):
        for symbol in self.symbols:
            self.cross_up[symbol] = False
    
    def reset_cross_down(self, symbols):
        for symbol in self.symbols:
            self.cross_down[symbol] = False



In [611]:
"""
BUY:
    Long: 
        1. Check if symbol crosses under lower boll. band
        2. Check if it goes back over (lower boll band * multiplier) -> Buy long

    Short:
        1. Check if symbol crosses over upper boll. band
        2. Check if it goes back under (upper boll band * 1/multiplier) -> Buy short

SELL:

1. 
"""

'\nBUY:\n    Long: \n        1. Check if symbol crosses under lower boll. band\n        2. Check if it goes back over (lower boll band * multiplier) -> Buy long\n\n    Short:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if it goes back under (upper boll band * 1/multiplier) -> Buy short\n\nSELL:\n\n1. \n'

In [612]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.balance_unit == 'BUSD' and env.short_unit == None:  # Want to buy
        for symbol in symbols:

            ######## LONG
            if env.cross_down[symbol] == True and df[f'{symbol}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] * 1/env.band_stop_run):
                env.buy_long(symbol, long_price=df[f'{symbol}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
                env.reset_cross_up(symbols)
                env.reset_cross_down(symbols)
                break
            
            if df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_Lower_Band'].iloc[i]:
                env.cross_down[symbol] = True
    

            ######## SHORT
            if env.cross_up[symbol] == True and df[f'{symbol}-USD_Close'].iloc[i] <= (df[f'{symbol}_Upper_Band'].iloc[i] * env.band_stop_run):
                env.buy_short(symbol, short_price=df[f'{symbol}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
                env.reset_cross_up(symbols)
                env.reset_cross_down(symbols)
                break
            
            if df[f'{symbol}-USD_High'].iloc[i] >= df[f'{symbol}_Upper_Band'].iloc[i]:
                env.cross_up[symbol] = True



    ####################################################################################################################

    elif env.balance_unit != 'BUSD':  # Want to sell long position
        ########### LONG
        if env.cross_up[env.balance_unit] == True and df[f'{env.balance_unit}-USD_Close'].iloc[i] <= (df[f'{env.balance_unit}_Upper_Band'].iloc[i] * env.band_stop_run):
            print('sell long')
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_cross_up(symbols)
            env.reset_cross_down(symbols)
            continue
        
        
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{env.balance_unit}_Upper_Band'].iloc[i]:
            print('long crossed')
            env.cross_up[env.balance_unit] = True
    


    else: # Want to sell short position
        ########### SHORT
        if env.cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_Close'].iloc[i] >= (df[f'{env.short_unit}_Lower_Band'].iloc[i] * 1/env.band_stop_run):
            print('sell short')
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_cross_up(symbols)
            env.reset_cross_down(symbols)
            continue

        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_Lower_Band'].iloc[i]:
            print('short crossed')
            env.cross_down[env.short_unit] = True





if env.balance_unit != 'BUSD':
    env.sell_long(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys)

100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
filled short
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
100 BUSD 1
short crossed
100 BUSD 1
short crossed
100 BUSD 1
sell short
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
99.82527427874847 BUSD 1
filled short
99.82527427874847 BUSD 2
99.82527427874847 BUSD 2
99.82527427874847 BUSD 2
99.82527427874847 BUSD 2
99.82527427874847 BUSD 2
99.82527427874847 BUSD 2
99.82527427874847 BUSD 2
99.82527427874847 BUSD 2
9

In [613]:

data = pd.DataFrame([(i[1] for i in env.buys), (j[2] for j in env.buys), (a[3] for a in env.buys), (k[1] for k in env.sells), (l[2] for l in env.sells)])
data

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,2020-12-29 19:22:00,2020-12-29 20:12:00,2020-12-29 20:36:00,2020-12-29 21:40:00,2020-12-29 22:31:00,2020-12-29 23:16:00,2020-12-29 23:52:00,2020-12-30 00:30:00,2020-12-30 00:38:00,2020-12-30 01:16:00,...,2021-01-26 13:57:00,2021-01-26 14:11:00,2021-01-26 14:42:00,2021-01-26 15:01:00,2021-01-26 15:35:00,2021-01-26 16:04:00,2021-01-26 16:43:00,2021-01-26 17:24:00,2021-01-26 17:34:00,2021-01-26 18:40:00
1,738.3,27658.95,737.5,739.05,27933.86,27772.61,27800.03,27906.77,27956.47,28045.77,...,1328.71,1353.08,32138.06,32201.6,31979.99,1335.8,32197.32,1329.45,32236.13,32606.6
2,short,short,long,short,long,long,long,long,short,short,...,long,short,short,short,long,long,long,long,short,long
3,2020-12-29 20:02:00,2020-12-29 20:30:00,2020-12-29 21:32:00,2020-12-29 22:29:00,2020-12-29 23:06:00,2020-12-29 23:40:00,2020-12-30 00:14:00,2020-12-30 00:34:00,2020-12-30 01:08:00,2020-12-30 02:16:00,...,2021-01-26 14:06:00,2021-01-26 14:25:00,2021-01-26 14:52:00,2021-01-26 15:26:00,2021-01-26 16:02:00,2021-01-26 16:32:00,2021-01-26 17:18:00,2021-01-26 17:31:00,2021-01-26 18:35:00,2021-01-26 19:00:59.999
4,739.59,27552.29,738.11,737.61,28113.96,27949.71,27938.21,27951.46,27964.99,28366.0,...,1344.27,1339.36,32150.02,32191.72,31941.48,1358.25,32098.27,1340.71,32676.52,32570.29
